Khi dữ liệu đầu vào có tỷ lệ khác nhau, tức là sử dụng các đơn vị đo khác nhau cho mỗi thuộc tính đầu vào. Vậy làm sao có thể đưa chúng về cùng một tỷ lệ, tức là bỏ qua đơn vị đo?

Câu trả lời giải quyết vấn đề là `scaling`.

Phương pháp chuẩn hóa (standardization):
$$x'=\frac{x - \mu}{\sigma}$$
Trong đó: $x$ là giá trị ban đầu, $x'$ là giá trị mới sau chuẩn hóa, $\mu$ là giá trị trung bình và $\sigma$ là độ lệch chuẩn.

Như vậy, trước hết cần phải tính được $\mu$ và $\sigma$ của cột đó (toàn bộ dữ liệu của đặc trưng).

Ví dụ, trong tập dữ liệu `cars.csv`, ở đặc trưng **weight** có giá trị trung bình là 1292.23, độ lệch chuẩn là 238.74. Suy ra giá trị 790 sẽ chuẩn hóa thành:
$$\frac{790-1292.23}{238.74}\approx -2.1$$

Tương tự, đặc trưng **volume** có giá trị trung bình là 1.61 và độ lệch chuẩn là 0.38. Giá trị 1 sẽ chuẩn hóa thành:
$$\frac{1.0-1.61}{0.38}\approx -1.59$$

Phương thức `StandardScaler()` trong thư viện `sklearn` sẽ đảm nhiệm công việc đó.

In [ ]:
# Tải các thư viện cần thiết
import pandas as pd
from sklearn import linear_model as lm
from sklearn.preprocessing import StandardScaler as ss

In [ ]:
scale = ss()

In [ ]:
# Tải dữ liệu về, lấy ra 5 dòng đầu tiên
data = pd.read_csv('https://www.w3schools.com/python/cars.csv')
print(data.head())

          Car       Model  Volume  Weight  CO2
0      Toyoty        Aygo    1000     790   99
1  Mitsubishi  Space Star    1200    1160   95
2       Skoda      Citigo    1000     929   95
3        Fiat         500     900     865   90
4        Mini      Cooper    1500    1140  105


In [ ]:
X = data[['Weight', 'Volume']]
y = data['CO2']

In [ ]:
scaled = scale.fit_transform(X)
print('Trước: \n', X[:5])
print('Sau: \n', scaled[:5])

Trước: 
    Weight  Volume
0     790    1000
1    1160    1200
2     929    1000
3     865     900
4    1140    1500
Sau: 
 [[-2.10389253 -1.59336644]
 [-0.55407235 -1.07190106]
 [-1.52166278 -1.59336644]
 [-1.78973979 -1.85409913]
 [-0.63784641 -0.28970299]]


In [ ]:
lr_1 = lm.LinearRegression()
lr_1.fit(X, y)
lr_2 = lm.LinearRegression()
lr_2.fit(scaled, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
print(lr_1.coef_)
print(lr_2.coef_)
print(lr_1.intercept_)
print(lr_2.intercept_)

[0.00755095 0.00780526]
[1.80269333 2.99358608]
79.69471929115937
102.02777777777777


Dễ nhận thấy các giá trị khác nhau của hai mô hình, vậy phải kiểm chứng lại.

In [ ]:
def f_1(weight, volume):
    return lr_1.intercept_ + lr_1.coef_[0] * weight + lr_1.coef_[1] * volume

def f_2(weight, volume):
    return lr_2.intercept_ + lr_2.coef_[0] * weight + lr_2.coef_[1] * volume

In [ ]:
# Lấy dữ liệu đầu tiên trước khi biến đổi
print(f_1(790, 1000))
# Lấy dữ liệu đầu tiên sau khi biến đổi
print(f_2(-2.10389253, -1.59336644))
# Nếu lấy dữ liệu trước khi biển đổi vào hàm thứ hai
print(f_2(790, 1000))

93.46522516244404
93.46522516073962
4519.741585094638


Hai dữ liệu chỉ khác nhau rất nhỏ, nguyên nhân là do trong `scale.fit_transform()` đã làm tròn dữ liệu. *Một điều lưu ý là đối với mô hình sử dụng dữ liệu đã được chuẩn hóa, dữ liệu đầu vào mới cần phải được chuẩn hóa theo đúng tỉ lệ.*